In [1]:
import requests
import pandas as pd
import os
import numpy as np

# Function to fetch data from the API
def fetch_data(url):
    try:
        response = requests.get(url)
        response.raise_for_status()  # Raise exception for HTTP errors
        return response.json()  # Return JSON data
    except requests.exceptions.RequestException as e:
        print(f"Request error: {e}")
        return []
    except ValueError as e:
        print(f"JSON decode error: {e}")
        return []

# API endpoints
travels_url = 'http://localhost:8000/list/'
checklists_url = 'http://localhost:8000/checklist/list/'

# Fetch data from the endpoints
travels_data = fetch_data(travels_url)
checklists_data = fetch_data(checklists_url)

# Convert to Pandas DataFrames
travels_df = pd.DataFrame(travels_data)
checklists_df = pd.DataFrame(checklists_data)

# Ensure IDs in both DataFrames are of the same type
if 'id' in travels_df.columns and 'id' in checklists_df.columns:
    travels_df['id'] = travels_df['id'].astype(float)
    checklists_df['id'] = checklists_df['id'].astype(float)

# Task 1: Return 500,000 rows in your dataset
if len(travels_df) < 500000:
    travels_df = pd.concat([travels_df] * (500000 // len(travels_df) + 1), ignore_index=True)[:500000]

# Task 2: Describe your dataset
print("Travels DataFrame Description:")
print(travels_df.describe(include='all'))

print("\nChecklists DataFrame Description:")
print(checklists_df.describe(include='all'))

# Task 3: Check and remove null values in your dataset

# Check for null values
print("\nNull Values in Travels DataFrame:")
print(travels_df.isnull().sum())

print("\nNull Values in Checklists DataFrame:")
print(checklists_df.isnull().sum())

# Remove rows with null values
travels_df.dropna(inplace=True)
checklists_df.dropna(inplace=True)

# Task 4: Perform basic data preprocessing

# Standardize text fields to lowercase
if 'name' in travels_df.columns:
    travels_df['name'] = travels_df['name'].str.lower()

if 'name' in checklists_df.columns:
    checklists_df['name'] = checklists_df['name'].str.lower()

# Normalize numerical columns
for col in travels_df.select_dtypes(include=[np.number]).columns:
    travels_df[col] = (travels_df[col] - travels_df[col].min()) / (travels_df[col].max() - travels_df[col].min())

# Task 5: Create some features in your dataset

# Create a new feature combining city and country
if 'city' in travels_df.columns and 'country' in travels_df.columns:
    travels_df['full_location'] = travels_df['city'] + ', ' + travels_df['country']

# Print the updated DataFrame heads
print("\nUpdated Travels DataFrame:")
print(travels_df.head())

print("\nUpdated Checklists DataFrame:")
print(checklists_df.head())

# Export individual DataFrames to CSV files
output_dir = "output_csvsquiz"
os.makedirs(output_dir, exist_ok=True)  

travels_csv_path = os.path.join(output_dir, "travels.csv")
checklists_csv_path = os.path.join(output_dir, "checklists.csv")

travels_df.to_csv(travels_csv_path, index=False)
checklists_df.to_csv(checklists_csv_path, index=False)

print(f"\nTravels DataFrame exported to {travels_csv_path}")
print(f"Checklists DataFrame exported to {checklists_csv_path}")

# Merge travels and checklists on 'id'
merged_df = pd.merge(
    travels_df,
    checklists_df,
    left_on='id',
    right_on='id',
    how='left',
    suffixes=('_travel', '_checklist')
)

# Print and export the merged DataFrame
print("\nMerged DataFrame:")
print(merged_df.head())

merged_csv_path = os.path.join(output_dir, "merged_data.csv")
merged_df.to_csv(merged_csv_path, index=False)

print(f"\nMerged DataFrame exported to {merged_csv_path}")


Travels DataFrame Description:
                   id           name  \
count   500000.000000         500000   
unique            NaN              5   
top               NaN  Shelby Daniel   
freq              NaN         100000   
mean         4.200000            NaN   
std          1.720467            NaN   
min          2.000000            NaN   
25%          3.000000            NaN   
50%          4.000000            NaN   
75%          5.000000            NaN   
max          7.000000            NaN   

                                                  country  \
count                                              500000   
unique                                                  5   
top     Consectetur qui laboriosam dolore fugiat quaer...   
freq                                               100000   
mean                                                  NaN   
std                                                   NaN   
min                                                   NaN   
